In [1]:

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import seaborn as sns
import re

import time, random, requests
from bs4 import BeautifulSoup

import ast
from pathlib import Path

In [2]:
file_path = r"./FINAL_dataset.xlsx"
data = pd.read_excel(file_path)
data.head()

,Ticker,TickerType,Company,Traded,Transaction,Trade_Size_USD,Status,Subholding,Description,Name,BioGuideID,Filed,Party,District,Chamber,Comments,Quiver_Upload_Time,excess_return,State,last_modified
0,AXP,ST,AMERICAN EXPRESS COMPANY COMMON STOCK,2025-05-05,Purchase,"$1,001 - $15,000",NEW,MARJORIE IRA,NaN,Marjorie Taylor Greene,G000596,2025-05-06,R,GA14,House,NaN,2025-05-07,2.203268,Georgia,2025-05-07
1,GILD,ST,"GILEAD SCIENCES, INC. - COMMON STOCK",2025-05-05,Purchase,"$1,001 - $15,000",NEW,NaN,NaN,Marjorie Taylor Greene,G000596,2025-05-06,R,GA14,House,NaN,2025-05-07,-5.866157,Georgia,2025-05-07
2,CMI,ST,CUMMINS INC. COMMON STOCK,2025-05-05,Purchase,"$1,001 - $15,000",NEW,MARJORIE IRA,NaN,Marjorie Taylor Greene,G000596,2025-05-06,R,GA14,House,NaN,2025-05-07,1.331400,Georgia,2025-05-07
3,AMAT,ST,"APPLIED MATERIALS, INC. - COMMON STOCK",2025-05-05,Purchase,"$1,001 - $15,000",NEW,MARJORIE IRA,NaN,Marjorie Taylor Greene,G000596,2025-05-06,R,GA14,House,NaN,2025-05-07,0.519373,Georgia,2025-05-07
4,GILD,ST,"GILEAD SCIENCES, INC. - COMMON STOCK",2025-05-05,Purchase,"$1,001 - $15,000",NEW,MARJORIE IRA,NaN,Marjorie Taylor Greene,G000596,2025-05-06,R,GA14,House,NaN,2025-05-07,-5.866157,Georgia,2025-05-07


In [3]:
data['TickerType'].unique()

array(['ST', 'GS', 'Stock', 'CT', 'Other Securities', nan, 'HN', 'OT',
       'ET', 'OP', 'Stock Option', 'OI', 'AB', 'CS', 'PS', 'SA', 'OL',
       'Cryptocurrency', 'Corporate Bond'], dtype=object)

In [4]:
#array(['Purchase', 'Sale', 'Sale (Full)', 'Sale (Partial)', 'Exchange'],
      #dtype=object)

In [5]:
# COLUMNS REMOVE: Company, Filed, Transaction (KEEP PURCHASE), Status, Subholding, 
#    Description, BioGuideID, Comments, Quiver_Upload_Time, last_modified
# COLUMNS NEED TO ADD: Sector (Stock), Politician Sector, Related
# RESULTING COLUMNS: Ticker, TickerType, Traded, Transaction, Trade_Size_USD, Name, 
#    Party, District, Chamber, excess_return, State, Sector (Stock), Politician Sector, Related 

# REMOVING UNNEEDED COLUMNS

#(TANNER WORK 6/10 2AM) --- 
# Filter to only 'Purchase' for Transaction
purchase_only_df = data[data['Transaction'] == 'Purchase'].copy()

# Dropping unnecessary columns
cols_to_drop = [
    'Company', 'Filed', 'Status', 'Subholding',
    'Description', 'BioGuideID', 'Comments',
    'Quiver_Upload_Time', 'last_modified'
]
purchase_only_df.drop(columns=cols_to_drop, inplace=True)

purchase_only_df.head()
#(TANNER WORK 6/10 2AM) --->


,Ticker,TickerType,Traded,Transaction,Trade_Size_USD,Name,Party,District,Chamber,excess_return,State
0,AXP,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,2.203268,Georgia
1,GILD,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,-5.866157,Georgia
2,CMI,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,1.331400,Georgia
3,AMAT,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,0.519373,Georgia
4,GILD,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,-5.866157,Georgia


In [6]:
print(data['Ticker'].nunique())

4128


In [7]:
unique_tickers = data['Ticker'].unique()
print(sorted(unique_tickers))

['07/01/28', '07/01/36', '0QZI.IL', '1.MONTH, MATURE', '12/01/37', '13.WEEK MATURE', '13.WEEK, MATURE', '2', '3%', '3.MONTH, MATURE', '3305779', '36966R5P7', '3V64.TI', '4.WEEK MATURE', '4.WEEK, MATURE', '40055AY43', '40056FN84', '4491235', '4824778', '506120KP8', '5522714', '6.MONTH', '6.MONTH, MATURE', '605699PU5', '6320058', '6897143', '784532JF1', '8.WEEK, MATURE', '912796CR8', '912796QV4', '912796QX0', '912796SH3', '912796U31', '912796U49', '912796WX3', '912796X87', '912796XQ7', '912796YH6', '912796YJ2', '912796ZN2', '912796ZP7', '912797KJ5', '912797LL9', '91282CDR9', '91282CEG2', '91282CFG1', '91282CFK2', '91282CFN6', '91282CJP7', '91282CJS1', '914476PV2', 'A', 'AA', 'AAGIY', 'AAIC$B', 'AAIGF', 'AAL', 'AAM$A', 'AAN', 'AAON', 'AAP', 'AAPL', 'AAT', 'AATC', 'AAVMY', 'AAXJ', 'AB', 'ABB', 'ABBNY', 'ABBV', 'ABC', 'ABCB', 'ABCO', 'ABDC', 'ABEV', 'ABG', 'ABM', 'ABMD', 'ABNB', 'ABNFX', 'ABQCX', 'ABT', 'ABX', 'ABYIX', 'ACA', 'ACAD', 'ACB', 'ACC', 'ACE', 'ACEIX', 'ACGL', 'ACGPF', 'ACHC', 'A

In [8]:
data[data['Ticker'] == '07/01/28']

,Ticker,TickerType,Company,Traded,Transaction,Trade_Size_USD,Status,Subholding,Description,Name,BioGuideID,Filed,Party,District,Chamber,Comments,Quiver_Upload_Time,excess_return,State,last_modified
28459,07/01/28,GS,METROPOLITAN ATLANTA RAPID TRAN AUTH,2019-05-09,Purchase,"$1,001 - $15,000",NEW,STIFEL,NaN,Earl Leroy Carter,C001103,2019-06-05,R,GA01,House,NaN,2020-07-26,NaN,Georgia,2023-11-16


In [9]:
# TICKER CLEANING BRAINSTORMING
# periods at end are not legal, slashes not legal, % not legal, cut all numbers (checked manually), cut carrots

#(TANNER WORK 6/10 2AM) --- 

#FIRST ROUND OF TICKER CLEANING (REMOVE NUMBERS, %, /, and ^)
# Function to validate tickers
def is_valid_ticker(ticker):
    if any(char in ticker for char in ['%', '/', '^']):
        return False
    if any(char.isdigit() for char in ticker):
        return False
    return True

# Create valid ticker list
valid_tickers = [ticker for ticker in unique_tickers if is_valid_ticker(ticker)]

# Filter dataframe to only keep valid tickers
purchase_only_df = purchase_only_df[purchase_only_df["Ticker"].isin(valid_tickers)].copy()

#(TANNER WORK 6/10 2AM) --- 

In [10]:
purchase_only_df

,Ticker,TickerType,Traded,Transaction,Trade_Size_USD,Name,Party,District,Chamber,excess_return,State
0,AXP,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,2.203268,Georgia
1,GILD,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,-5.866157,Georgia
2,CMI,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,1.331400,Georgia
3,AMAT,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,0.519373,Georgia
4,GILD,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,-5.866157,Georgia
...,...,...,...,...,...,...,...,...,...,...,...
51283,ABBV,NaN,2013-01-02,Purchase,"$1,001 - $15,000",Ms. Tammy Duckworth,D,IL08,House,137.415346,Illinois
51285,DD,Stock,2012-12-17,Purchase,"$1,001 - $15,000","Carper, Thomas R.",D,NaN,Senate,-343.822825,Delaware
51287,DD,Stock,2012-09-13,Purchase,"$1,001 - $15,000","Carper, Thomas R.",D,NaN,Senate,-342.945845,Delaware
51289,CAT,NaN,2012-07-26,Purchase,"$1,001 - $15,000",Ms. Tammy Duckworth,D,IL08,House,-25.552185,Illinois


In [11]:
tickers = purchase_only_df['Ticker'].dropna().unique().tolist()

In [12]:
len(tickers)

3074

In [13]:
temp = purchase_only_df.copy(deep=True)

In [14]:
purchase_only_df = temp.copy(deep=True)

# Files to search, in order

no_prefix_files = {"usa.csv", "us-otc.csv"}

# TODO: FIX PATH LATER
prefix = "../TICKERS/"

purchase_only_df["Sector"] = None
purchase_only_df["Ticker"] = purchase_only_df["Ticker"].astype(str)

# Track which tickers are still unmatched
unmatched_mask = purchase_only_df["Sector"].isna()

import pandas as pd

# List of datasets and which ones have prefixes
csv_files = [
    "usa.csv",        # no prefix
    "us-otc.csv",     # no prefix
    "uk.csv",         # has prefix
    "japan.csv",      # has prefix
    "canada.csv",     # has prefix
    "hongkong.csv",   # has prefix
    "china.csv",      # has prefix
    "australia.csv",  # has prefix
    "india.csv",      # has prefix
    "germany.csv",    # has prefix
    "southkorea.csv",# has prefix
    "taiwan.csv"      # has prefix
]

# Files that do NOT have exchange prefixes in the ticker symbol
no_prefix_files = {"usa.csv", "us-otc.csv"}

# Keep track of tickers still needing matches
unmatched_mask = purchase_only_df["Sector"].isna()

# Process each dataset
for file in csv_files:
    try:
        df = pd.read_csv(prefix+file)
    except Exception as e:
        print(f"Error reading {file}: {e}")
        continue

    df["Symbol"] = df["Symbol"].astype(str)

    # If this file uses prefixes, extract the base ticker
    if file not in no_prefix_files:
        # Assume format like "LSE:BP" or "JPX:7203"
        df["Clean Ticker"] = df["Symbol"].str.split(":").str[-1]
        ticker_col = "Clean Ticker"
    else:
        ticker_col = "Symbol"

    # Create mapping from cleaned ticker to sector
    ticker_to_sector = df.set_index(ticker_col)["Sector"].to_dict()

    # Apply mapping to unmatched tickers
    purchase_only_df.loc[unmatched_mask, "Sector"] = purchase_only_df.loc[unmatched_mask, "Ticker"].map(ticker_to_sector)

    # Update unmatched mask
    unmatched_mask = purchase_only_df["Sector"].isna()
    if not unmatched_mask.any():
        break

# Save the results
purchase_only_df.to_excel("trading_with_sectors.xlsx", index=False)
print("Matching complete. Saved to trading_with_sectors.xlsx")


Matching complete. Saved to trading_with_sectors.xlsx


In [15]:
traded_with_sectors = pd.read_excel("./trading_with_sectors.xlsx")

In [16]:
etf_df = pd.read_csv(prefix + "etf.csv")
etf_df["Symbol"] = etf_df["Symbol"].astype(str)

# Get list of ETF tickers
etf_tickers = set(etf_df["Symbol"])

# Find unmatched rows
unmatched_mask = traded_with_sectors["Sector"].isna()

# Find which unmatched tickers are in the ETF list
is_etf = traded_with_sectors["Ticker"].isin(etf_tickers)

# Set Sector to "ETF" where ticker is in ETF list and was previously unmatched
traded_with_sectors.loc[unmatched_mask & is_etf, "Sector"] = "ETF"

traded_with_sectors.to_excel("traded_etf.xlsx", index=False)

In [17]:
traded_final = pd.read_excel("traded_etf.xlsx")

In [18]:
traded_final.dropna(subset=["Sector", "excess_return"], inplace=True)

In [19]:
traded_final.to_excel("traded_final.xlsx", index=False)

In [20]:
traded_final

,Ticker,TickerType,Traded,Transaction,Trade_Size_USD,Name,Party,District,Chamber,excess_return,State,Sector
0,AXP,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,2.203268,Georgia,Financials
1,GILD,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,-5.866157,Georgia,Healthcare
2,CMI,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,1.331400,Georgia,Industrials
3,AMAT,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,0.519373,Georgia,Technology
4,GILD,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,-5.866157,Georgia,Healthcare
...,...,...,...,...,...,...,...,...,...,...,...,...
25269,KMI,NaN,2013-01-15,Purchase,"$1,001 - $15,000",Mr. Beto O'Rourke,D,TX16,House,-311.023827,Texas,Energy
25271,ABBV,NaN,2013-01-02,Purchase,"$1,001 - $15,000",Ms. Tammy Duckworth,D,IL08,House,137.415346,Illinois,Healthcare
25272,DD,Stock,2012-12-17,Purchase,"$1,001 - $15,000","Carper, Thomas R.",D,NaN,Senate,-343.822825,Delaware,Materials
25273,DD,Stock,2012-09-13,Purchase,"$1,001 - $15,000","Carper, Thomas R.",D,NaN,Senate,-342.945845,Delaware,Materials


In [21]:
# MATCH CONGRESSMAN SECTORS
# Load the Excel file
politicians = pd.read_excel("POLITICIANS_AND_COMMITTEES.xlsx")  # Replace with your actual filename
politicians

,politician_name,committees
0,Abigail Spanberger,"[""Agriculture"", ""Foreign Affairs""]"
1,April McClain Delaney,"[""Agriculture"", ""Science, Space, and Technology""]"
2,Ashley Moody,"[""Health, Education, Labor, and Pensions"", ""Ho..."
3,August Pfluger,"[""Homeland Security"", ""Foreign Affairs"", ""Ener..."
4,Bill Keating,"[""Armed Services"", ""Foreign Affairs""]"
...,...,...
416,William M. Cassidy,"[""Energy and Commerce""]"
417,William R. Keating,"[""Armed Services"", ""Foreign Affairs""]"
418,William R. Timmons Iv,"[""Financial Services"", ""Oversight and Accounta..."
419,"Wyden, Ron","[""Budget"", ""Energy and Natural Resources"", ""Fi..."


In [22]:
sector_dict = {
    "Administration": "no_sector",
    "Aging": "no_sector",
    "Agriculture": "Agriculture",
    "Agriculture and Natural Resources": "Agriculture",
    "Agriculture, Environment and Natural Resources": "Agriculture",
    "Agriculture, Nutrition and Forestry": "Agriculture",
    "Agriculture, Nutrition, & Forestry": "Agriculture",
    "Agriculture, Nutrition, and Forestry": "Agriculture",
    "Appropriations": "no_sector",
    "Armed Services": "Industrials",
    "Armed Services ": "Industrials",
    "Banking": "Financials",
    "Banking Housing, and Urban Affairs": "Financials",
    "Banking, Housing, and Urban Affairs": "Financials",
    "Banks": "Financials",
    "Benghazi": "no_sector",
    "Budget": "no_sector",
    "Capital Budget": "no_sector",
    "Caucus memberships": "no_sector",
    "Children, Youth, and Families": "no_sector",
    "Chinese Communist Party": "no_sector",
    "Climate Crisis": "no_sector",
    "Codes": "no_sector",
    "Commerce": "no_sector",
    "Commerce, Science and Transportation": "Industrials",
    "Commerce, Science, and Transportation": "Industrials",
    "Commission on Security and Cooperation in Europe": "no_sector",
    "Company's Compensation": "Financials",
    "Coronavirus Crisis": "no_sector",
    "Corporations, Authorities, and Commissions": "no_sector",
    "Correction": "no_sector",
    "Defense": "Industrials",
    "East Moline Downtown Revitalization": "Real Estate",
    "Economic Development, Trade, & Innovation": "no_sector",
    "Education": "no_sector",
    "Education and Labor": "no_sector",
    "Education and the Workforce": "no_sector",
    "Energy and Commerce": "Energy",
    "Energy and Mineral Resources": "Energy",
    "Energy and Natural Resources": "Energy",
    "Energy and Water Development, and Related Agencies": "Energy",
    "Environment and Public Works": "Utilities",
    "Environmental Quality": "Utilities",
    "Ethics": "no_sector",
    "Finance": "Financials",
    "Financial Institutions": "Financials",
    "Financial Services": "Financials",
    "Foreign Affairs": "no_sector",
    "Foreign Affairs ": "no_sector",
    "Foreign Relations": "no_sector",
    "Government Operations": "no_sector",
    "Health Care": "Healthcare",
    "Health Care Access and Affordability": "Healthcare",
    "Health Education Labor & Pensions": "Healthcare",
    "Health, Education, Labor and Pensions": "Healthcare",
    "Health, Education, Labor, and Pension": "Healthcare",
    "Health, Education, Labor, and Pensions": "Healthcare",
    "Higher Education": "no_sector",
    "Higher Education & Workforce Development": "no_sector",
    "Homeland Security": "Industrials",
    "Homeland Security and Governmental Affairs": "Industrials",
    "House Administration": "no_sector",
    "House Budget": "no_sector",
    "House Democratic Steering and Policy": "no_sector",
    "House Rules": "no_sector",
    "Housing": "Real Estate",
    "Indian Affairs": "no_sector",
    "Indian Affairs ": "no_sector",
    "Insurance": "Financials",
    "Insurance and Commerce": "Financials",
    "Intelligence": "no_sector",
    "Intelligence ": "no_sector",
    "International Relations": "no_sector",
    "January 6 Attack": "no_sector",
    "January 6th Attack": "no_sector",
    "Joint Deficit Reduction": "no_sector",
    "Joint Economic": "no_sector",
    "Joint Library": "no_sector",
    "Joint Taxation": "Financials",
    "Judiciary": "no_sector",
    "Labor and Human Resources": "no_sector",
    "Labor, Health and Human Services, Education and Related Agencies": "Healthcare",
    "Legislative Budget": "no_sector",
    "Legislative Oversight": "no_sector",
    "Legislative Services Commission": "no_sector",
    "Merchant Marine and Fisheries": "Industrials",
    "Modernization of Congress": "no_sector",
    "Natural Resources": "Materials",
    "Natural Resources and Environmental Affairs": "Materials",
    "Oversight": "no_sector",
    "Oversight and Accountability": "no_sector",
    "Oversight and Government Reform": "no_sector",
    "Oversight and Investigations": "no_sector",
    "Oversight and Reform": "no_sector",
    "President's Education Policy Advisory": "no_sector",
    "Primary Health and Retirement Security": "Healthcare",
    "Program Evaluation": "no_sector",
    "Public Works": "Industrials",
    "Real Estate and Securities": "Real Estate",
    "Republican National": "no_sector",
    "Revenue": "Financials",
    "Revenue And Taxation": "Financials",
    "Rules": "no_sector",
    "Rules and Administration": "no_sector",
    "Science, Space, and Technology": "Technology",
    "Security and Cooperation in Europe": "no_sector",
    "Select Intelligence": "no_sector",
    "Small Business": "no_sector",
    "Small Business and Entrepreneurship": "no_sector",
    "Special Aging": "no_sector",
    "State Agencies And Governmental Affairs": "no_sector",
    "Strategic Competition": "no_sector",
    "Strategic Competition between the United States and the Chinese Communist Party": "no_sector",
    "Strategic Competition between the United States and the Chinese Communist Party ": "no_sector",
    "Taxation": "Financials",
    "Transportation": "Industrials",
    "Transportation & Infrastructure": "Industrials",
    "Transportation and Infrastructure": "Industrials",
    "UNC Board of Governors": "no_sector",
    "Veterans Affairs": "Healthcare",
    "Veterans' Affairs": "Healthcare",
    "Way and Means": "Financials",
    "Ways & Means": "Financials",
    "Ways and Means": "Financials",
    "the Judiciary": "no_sector"
}

In [23]:
traded_final

,Ticker,TickerType,Traded,Transaction,Trade_Size_USD,Name,Party,District,Chamber,excess_return,State,Sector
0,AXP,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,2.203268,Georgia,Financials
1,GILD,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,-5.866157,Georgia,Healthcare
2,CMI,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,1.331400,Georgia,Industrials
3,AMAT,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,0.519373,Georgia,Technology
4,GILD,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,-5.866157,Georgia,Healthcare
...,...,...,...,...,...,...,...,...,...,...,...,...
25269,KMI,NaN,2013-01-15,Purchase,"$1,001 - $15,000",Mr. Beto O'Rourke,D,TX16,House,-311.023827,Texas,Energy
25271,ABBV,NaN,2013-01-02,Purchase,"$1,001 - $15,000",Ms. Tammy Duckworth,D,IL08,House,137.415346,Illinois,Healthcare
25272,DD,Stock,2012-12-17,Purchase,"$1,001 - $15,000","Carper, Thomas R.",D,NaN,Senate,-343.822825,Delaware,Materials
25273,DD,Stock,2012-09-13,Purchase,"$1,001 - $15,000","Carper, Thomas R.",D,NaN,Senate,-342.945845,Delaware,Materials


In [24]:
def map_committees_to_sectors(committee_list_str, mapping_dict):
    """
    Maps a list of committees (as a string representation of a list) to their
    corresponding sectors using the provided mapping dictionary.
    Handles potential errors and ensures unique sectors are returned.
    """
    sectors = set()
    try:
        # Safely evaluate the string representation of the list
        committees = ast.literal_eval(committee_list_str)
        if not isinstance(committees, list):
            # If it's not a list after evaluation, treat it as a single committee string
            committees = [committees]
    except (ValueError, SyntaxError):
        # If evaluation fails, treat the original string as a single committee
        committees = [committee_list_str]

    for committee in committees:
        sector = mapping_dict.get(committee.strip(), "no_sector") # .strip() removes whitespace
        sectors.add(sector)
    return sorted(list(sectors)) # Return sorted unique sectors as a list

file_path = 'your_file.xlsx'

try:
    politicians['sectors'] = politicians['committees'].apply(
        lambda x: map_committees_to_sectors(str(x), sector_dict)
    )

    # Display the first few rows of the updated DataFrame
    print("Updated DataFrame (first 5 rows):")
    print(politicians.head())

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found. Please check the path.")
except KeyError as e:
    print(f"Error: Missing expected column. Make sure your XLSX has 'politician_name' and 'committees' columns. Detail: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Updated DataFrame (first 5 rows):
         politician_name                                         committees  \
0     Abigail Spanberger                 ["Agriculture", "Foreign Affairs"]   
1  April McClain Delaney  ["Agriculture", "Science, Space, and Technology"]   
2           Ashley Moody  ["Health, Education, Labor, and Pensions", "Ho...   
3         August Pfluger  ["Homeland Security", "Foreign Affairs", "Ener...   
4           Bill Keating              ["Armed Services", "Foreign Affairs"]   

                                sectors  
0              [Agriculture, no_sector]  
1             [Agriculture, Technology]  
2  [Healthcare, Industrials, no_sector]  
3      [Energy, Industrials, no_sector]  
4              [Industrials, no_sector]  


In [25]:
politicians

,politician_name,committees,sectors
0,Abigail Spanberger,"[""Agriculture"", ""Foreign Affairs""]","[Agriculture, no_sector]"
1,April McClain Delaney,"[""Agriculture"", ""Science, Space, and Technology""]","[Agriculture, Technology]"
2,Ashley Moody,"[""Health, Education, Labor, and Pensions"", ""Ho...","[Healthcare, Industrials, no_sector]"
3,August Pfluger,"[""Homeland Security"", ""Foreign Affairs"", ""Ener...","[Energy, Industrials, no_sector]"
4,Bill Keating,"[""Armed Services"", ""Foreign Affairs""]","[Industrials, no_sector]"
...,...,...,...
416,William M. Cassidy,"[""Energy and Commerce""]",[Energy]
417,William R. Keating,"[""Armed Services"", ""Foreign Affairs""]","[Industrials, no_sector]"
418,William R. Timmons Iv,"[""Financial Services"", ""Oversight and Accounta...","[Financials, no_sector]"
419,"Wyden, Ron","[""Budget"", ""Energy and Natural Resources"", ""Fi...","[Energy, Financials, no_sector]"


In [26]:
politicians.to_excel("politicians_with_sectors.xlsx", index=False)

In [27]:
full_politicians = pd.read_excel("./politicians_with_sectors.xlsx")

In [28]:
full_politicians

,politician_name,committees,sectors
0,Abigail Spanberger,"[""Agriculture"", ""Foreign Affairs""]","['Agriculture', 'no_sector']"
1,April McClain Delaney,"[""Agriculture"", ""Science, Space, and Technology""]","['Agriculture', 'Technology']"
2,Ashley Moody,"[""Health, Education, Labor, and Pensions"", ""Ho...","['Healthcare', 'Industrials', 'no_sector']"
3,August Pfluger,"[""Homeland Security"", ""Foreign Affairs"", ""Ener...","['Energy', 'Industrials', 'no_sector']"
4,Bill Keating,"[""Armed Services"", ""Foreign Affairs""]","['Industrials', 'no_sector']"
...,...,...,...
416,William M. Cassidy,"[""Energy and Commerce""]",['Energy']
417,William R. Keating,"[""Armed Services"", ""Foreign Affairs""]","['Industrials', 'no_sector']"
418,William R. Timmons Iv,"[""Financial Services"", ""Oversight and Accounta...","['Financials', 'no_sector']"
419,"Wyden, Ron","[""Budget"", ""Energy and Natural Resources"", ""Fi...","['Energy', 'Financials', 'no_sector']"


In [29]:
try:
    full_politicians['sectors'] = full_politicians['sectors'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )
except (ValueError, SyntaxError):
    print("Warning: 'sectors' column in full_politicians might not be in a safe list format for all rows. Proceeding without `ast.literal_eval`.")


# --- 3. Perform the merge operation ---
# We'll use a left merge to keep all rows from traded_final.
# We join on 'Name' from traded_final and 'politician_name' from full_politicians.
traded_final = pd.merge(
    traded_final,
    full_politicians[['politician_name', 'sectors']], # Select only necessary columns from full_politicians
    left_on='Name',                                  # Column from traded_final
    right_on='politician_name',                      # Column from full_politicians
    how='left'                                       # Keep all rows from traded_final
)

traded_final.rename(columns={'sectors': 'politician_sectors'}, inplace=True)

traded_final.drop(columns=['politician_name'], inplace=True)

print("\nUpdated traded_final DataFrame with 'politician_sectors':")


Updated traded_final DataFrame with 'politician_sectors':


In [30]:
traded_final

,Ticker,TickerType,Traded,Transaction,Trade_Size_USD,Name,Party,District,Chamber,excess_return,State,Sector,politician_sectors
0,AXP,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,2.203268,Georgia,Financials,"[Industrials, no_sector]"
1,GILD,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,-5.866157,Georgia,Healthcare,"[Industrials, no_sector]"
2,CMI,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,1.331400,Georgia,Industrials,"[Industrials, no_sector]"
3,AMAT,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,0.519373,Georgia,Technology,"[Industrials, no_sector]"
4,GILD,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,-5.866157,Georgia,Healthcare,"[Industrials, no_sector]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21554,KMI,NaN,2013-01-15,Purchase,"$1,001 - $15,000",Mr. Beto O'Rourke,D,TX16,House,-311.023827,Texas,Energy,"[Healthcare, Industrials]"
21555,ABBV,NaN,2013-01-02,Purchase,"$1,001 - $15,000",Ms. Tammy Duckworth,D,IL08,House,137.415346,Illinois,Healthcare,"[Industrials, no_sector]"
21556,DD,Stock,2012-12-17,Purchase,"$1,001 - $15,000","Carper, Thomas R.",D,NaN,Senate,-343.822825,Delaware,Materials,"[Financials, Industrials, Utilities]"
21557,DD,Stock,2012-09-13,Purchase,"$1,001 - $15,000","Carper, Thomas R.",D,NaN,Senate,-342.945845,Delaware,Materials,"[Financials, Industrials, Utilities]"


In [31]:
# Initialize 'trade_related' column with 0
traded_final['trade_related'] = 0

# Apply a function to each row to check for sector match
def check_trade_related(row):
    # Normalize trade_sector for comparison (lowercase and strip whitespace)
    trade_sector = str(row['Sector']).strip().lower()
    politician_name = row['Name'] # For debugging prints

    politician_sectors_list = row['politician_sectors']

    # IMPORTANT FIX: Handle cases where 'politician_sectors_list' might unexpectedly be a pandas Series
    if isinstance(politician_sectors_list, pd.Series):
        if not politician_sectors_list.empty:
            # If it's a non-empty Series, extract the first element (assuming it should be scalar)
            politician_sectors_list = politician_sectors_list.iloc[0]
        else:
            # If it's an empty Series, treat it as NaN
            politician_sectors_list = np.nan




    # Ensure politician_sectors_list is a list before processing
    if isinstance(politician_sectors_list, list):
        # Normalize each sector in the politician's list
        normalized_politician_sectors = [str(s).strip().lower() for s in politician_sectors_list]

        # Check if the normalized trade_sector is in the normalized politician's sectors
        if trade_sector in normalized_politician_sectors:
            return 1
        else:
            return 0
    elif pd.isna(politician_sectors_list): # Explicitly check for NaN using pandas.isna
        return 0
    else:
        # Handle cases where politician_sectors_list is neither a list nor NaN (unexpected type)
        return 0

# Apply the function. The output of the print statements will show in your console.
traded_final['trade_related'] = traded_final.apply(check_trade_related, axis=1)

# --- Display the final updated DataFrame ---
print("\n" + "="*50)
print("Final Updated traded_final DataFrame:")
print(traded_final.to_string())
print("="*50 + "\n")


Final Updated traded_final DataFrame:


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [32]:
traded_final

,Ticker,TickerType,Traded,Transaction,Trade_Size_USD,Name,Party,District,Chamber,excess_return,State,Sector,politician_sectors,trade_related
0,AXP,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,2.203268,Georgia,Financials,"[Industrials, no_sector]",0
1,GILD,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,-5.866157,Georgia,Healthcare,"[Industrials, no_sector]",0
2,CMI,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,1.331400,Georgia,Industrials,"[Industrials, no_sector]",1
3,AMAT,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,0.519373,Georgia,Technology,"[Industrials, no_sector]",0
4,GILD,ST,2025-05-05,Purchase,"$1,001 - $15,000",Marjorie Taylor Greene,R,GA14,House,-5.866157,Georgia,Healthcare,"[Industrials, no_sector]",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21554,KMI,NaN,2013-01-15,Purchase,"$1,001 - $15,000",Mr. Beto O'Rourke,D,TX16,House,-311.023827,Texas,Energy,"[Healthcare, Industrials]",0
21555,ABBV,NaN,2013-01-02,Purchase,"$1,001 - $15,000",Ms. Tammy Duckworth,D,IL08,House,137.415346,Illinois,Healthcare,"[Industrials, no_sector]",0
21556,DD,Stock,2012-12-17,Purchase,"$1,001 - $15,000","Carper, Thomas R.",D,NaN,Senate,-343.822825,Delaware,Materials,"[Financials, Industrials, Utilities]",0
21557,DD,Stock,2012-09-13,Purchase,"$1,001 - $15,000","Carper, Thomas R.",D,NaN,Senate,-342.945845,Delaware,Materials,"[Financials, Industrials, Utilities]",0


In [33]:
traded_final.value_counts("trade_related")

trade_related
0    18981
1     2578
Name: count, dtype: int64

In [34]:
traded_final.to_excel("FINAL_related_dataset.xlsx", index=False)